In [3]:
import pandas as pd
import numpy as np
import re
import spacy

In [4]:
pd.set_option('display.max_colwidth', 1000)

In [5]:
all_amz_go = pd.read_csv('F:\\mailae_g\\ALL_MAG_2021+2022.csv', sep=';' )
all_cus_go = pd.read_csv('F:\\mailae_g\\ALL_PL_2022.csv', sep=';')

In [6]:
all_amz_go.head(5)

,Ticket Id,Subject,Description,Source,Requester Email,Created Time,Agent interactions,Customer interactions,Tags
0,16477,Frage von Amazon-Kunde Anja Klose,Skraut Home- Glanzlack Wohnwand. Von dem großen hängeschrank der Boden ist beschädigt. Beim Dübel loch ist was abgebrochen.,Email,freshdeskuser_18653026@channelreply.email,13 09 2021 15:58,26,18,"lukfurniture_de,cr,lukfurniture,rynek_de"
1,83046,Frage von Amazon-Kunde Furkan Aydin(Bestellung: 306-1709843-3097134),Hallo Die schiene von der schublade ist kaputt da fallen die kleinen Kugeln raus. Ich möchte gerne 2 schienen.,Email,freshdeskuser_20098643@channelreply.email,7 12 2021 22:56,12,6,"lukfurniture_de,cr,lukfurniture,rynek_de"
2,92323,Return policy enquiry from Amazon customer Mrs Susan Tolley(Order: 204-9663987-5997940),Hi I purchased the welder back in April 2021 but due to ill health I have only been able to try the welder today. Unfortunately It does not work. I think there maybe a problem with the MIG output to the switch on the gun. I have used a test meter on the contacts and have no output. Please can you advise. Thankyou.,Email,freshdeskuser_20274812@channelreply.email,16 12 2021 13:51,65,40,"cr,rynek_uk,allweld,allweld_uk"
3,165940,Frage von Amazon-Kunde Fjolla(Bestellung: 306-7063158-3705902),Sehr Geehrte Damen und Herren Es geht um meine Bestellung die ich seit Wochen gemacht habe.Auf meine Bestellung steht dass mein Paket möglicherweise verloren gegangen ist.Ich habe dafür Geld bezahlt.Wie können sie mir weiter helfen? Mit Freundliche Grüßen,Email,freshdeskuser_21788405@channelreply.email,19 03 2022 17:47,14,9,"lukfurniture_de,cr,lukfurniture,rynek_de"
4,192109,"? Sie haben einen neuen Preisvorschlag: EUR 60,00 für NeoNail Nagelstudio ... (373940992894)","Preisvorschlag: EUR 60,00 Preisvorschlag: EUR 60,00 Jetzt antworten: https://www.ebay.de/ulk/showOffers/373940992894?boolp=1&amp;mkevt=1&amp;mkpid=0&amp;emsid=e12060.m3112.l44861&amp;mkcid=26&amp;ch=osgood&amp;euid=0fe5676504234b2c849a719471b7060f&amp;bu=45010273334&amp;osub=-1%7E1&amp;crd=20220424091836&amp;segname=12060&amp;sojTags=ch%3Dch%2Cbu%3Dbu%2Cosub%3Dosub%2Ccrd%3Dcrd%2Csegname%3Dsegname%2Cchnl%3Dmkcid Artikel ansehen: https://www.ebay.de/itm/373940992894?mkevt=1&amp;mkpid=0&amp;emsid=e12060.m3112.l44862&amp;mkcid=26&amp;ch=osgood&amp;euid=0fe5676504234b2c849a719471b7060f&amp;bu=45010273334&amp;osub=-1%7E1&amp;crd=20220424091836&amp;segname=12060&amp;sojTags=ch%3Dch%2Cbu%3Dbu%2Cosub%3Dosub%2Ccrd%3Dcrd%2Csegname%3Dsegname%2Cchnl%3Dmkcid Unter Angebotspreis EUR 4,89 Dieses Angebot sichern 9 Beobachter meliba601: https://www.ebay.de/ulk/usr/meliba601?mkevt=1&amp;mkpid=0&amp;emsid=e12060.m3112.l44863&amp;mkcid=26&amp;ch=osgood&amp;euid=0fe5676504234b2c849a719471b7060f&amp;bu=4...",Email,freshdeskuser_22171370@channelreply.email,24 04 2022 18:56,1,12,"ServiceMessage,cr,gashio,gashio_de_eb,ebay,rynek_de_eb"


In [7]:
all_cus_go.head(5)

,Ticket Id,Subject,Description,Source,Requester Email,Created Time,Agent interactions,Customer interactions,Tags
0,111345,[DE] [PYTANIE] B07R42W8J9,Klient ma pytanie do produktu B07R42W8J9: &quot;Jakie połączenie ma dachówka wentylacyjna?&quot; Link do zapytania: www.amazon.de/ask/question/Tx21NZNJ8PMI5JH?_encoding=UTF8&amp;authToken=967c969c-3f2e-47dd-b5e6-faeb39d052d1&amp;ref_=s_respond&amp;answerSolicitationSourceType=EMAIL&amp;answerSolicitationLanguage=de_DE Pozdrawiam Beata,Outbound Email,shop@folnet.de,10 01 2022 10:47,4,4,"Partner_wysłane,rynek_de"
1,128291,[DE] 302-0014409-7188317 NIEKOMPLETNE,"Klient zgłasza, że w zamówieniu nie było lampki. Pozdrawiam, Jakub",Outbound Email,info.palulli@gmail.com,30 01 2022 09:48,3,2,"Partner_wysłane,rynek_de"
2,128325,[REKLAMACJA] 205-9594094-3558723 UK,"Dzień dobry, Klient napisał: Ten produkt dotarł, ale nie działa, nie świeci się. Bardzo rozczarowany, ponieważ kupiłem go dla mojego dziecka. Brak zauważalnych wad, wszystko wydawało się w porządku po otwarciu. Mały przycisk przesuwa się w dół i wydaje się OK, ale nic się nie dzieje, gdy go nacisnąć. To nie świeci się, co było cały punkt z nim, i bardzo rozczarowujące dla mojej córki. Pozdrawiam, Hania",Outbound Email,polskaksiegarniainternetowa@gmail.com,30 01 2022 10:38,2,2,"Partner_wysłane,rynek_uk,Przypomnienie"
3,131092,[DE] [ZAPYTANIE] 305-6969626-5165121,"Klient chciałby wiedzieć, czy materiał, z którego wykonane są klamki może być stosowany również na zewnątrz, np. na bocznych drzwiach wejściowych. Pozdrawiam Beata",Outbound Email,kotarbauzapytania@gmail.com,2 02 2022 16:10,4,4,"Partner_wysłane,rynek_de,Przypomnienie"
4,133983,[IT] 408-8392161-3066735 ZWROT,"https://www.dhl.com/it-en/home/tracking/tracking-parcel.html?submit=1&amp;tracking-id=CM332431606DE Paczka wraca do was, prosimy o refund dla klienta jak dojdzie Kasia",Outbound Email,kotarbauzapytania@gmail.com,7 02 2022 08:58,9,6,"Partner_wysłane,rynek_it,Przypomnienie1,Przypomnienie2"


In [8]:
all_amz_go.reset_index(drop=True, inplace=True)
all_cus_go.reset_index(drop=True, inplace=True)

In [9]:
# Funkcja do dzielenia komórek na listę, pierwszy argument tekst, drugi argument po czym tekst ma być dzielony 

def split_tags(tags: str, split_char: str) -> list:
    try:
        tags = tags.split(f'{split_char}')
        return tags 
    except Exception:
        return (list[tags])
    

In [10]:
# funkcja do wyciągania z listy, jakiego rynku dotyczą wiadomości

def get_market(lst: list) -> str:
    for i in lst:
        if "rynek" in i:
            return i

In [11]:
# funkcja do wyciągania numerów zamówień z tekstu i tytułu maila. Przyjmuje dwa różne ciągi tekstowe, sprawdza czy jest w tytule jest regex, jeśli nie to sprawdza czy jest w opisie, jeśli nie to zwraca pustą listę

se = re.compile(r"\d{3}-\d{7}-\d{7}")

def order_number(subject: str, description: str) -> list:
    subject = str(subject)
    description = str(description)
    if se.findall(subject) == []:
        return se.findall(description)
    return se.findall(subject)


In [12]:
# funkcja do tworzenia listy unikalnych wartości z kolumn. Dajemy wartość z kolumny i listę. Sprawdza czy wartość już znajduje się na liście, jeśli nie to ją dodaje

def all_order(order: list, new_list: list) -> list:
    try:
        if order[0] in new_list:
            return new_list
        else:
            new_list.append(order)
            return new_list
    except Exception:
        i =1

In [13]:
# druga funkcja do czyszczenia tytułów w celu zebrania etykiet do danych

on = re.compile(r"\d{3}-\d{7}-\d{7}")
cc = re.compile(r"w{2}")
cc2 = re.compile(r'\[\w{2}\]')

def clear_title(title_list: list) -> list:
    for el in title_list:
        print(el)
    list_to_remove = []
    for i in range(len(title_list)):
        if re.match(on, title_list[i]):
            title_list.remove(i)
        elif re.match(cc, title_list[i]):
            title_list.remove(i)
        elif re.match(cc2, title_list[i]):
            title_list.remove(i)
    return title_list

In [14]:
# trzecia funkcja do czyszczenia tytułów w celu zebrania etykiet do danych

def num_there(s):
    return any(i.isdigit() for i in s)

def find_words_with_more_than_four_characters(words_with_tags):
  words = []
  for i in words_with_tags:
    if num_there(i):
      continue
    w = re.sub('[^A-Za-zĄĆĘŁŃÓŚŹŻąćęłńóśźż]+', ' ', i)
    w = w.strip(" ")
    if len(w) >= 4:
      words.append(w)

  return words

In [15]:
# czwarta funkcja do czyszczenia tytułów w celu zebrania etykiet do danych

def find_words_with_more_than_four_characters2(subject: str) -> list:
  words = []
  new_subject = ''
  for l in subject:
    l = re.sub('[^A-Za-zĄĆĘŁŃÓŚŹŻąćęłńóśźż]+', ' ', l)
    new_subject += l
  words_temp = new_subject.split(' ')
  for i in words_temp:
    if len(i) >= 4:
      words.append(i)

  return words

In [16]:
# Funkcja do zliczania elementów w liscie, zwraca listę unikatowych wartości i listę unikatowych wartości z informacją ile razy występowały

from collections import Counter

def count_occurrences(input_list):
    unique_values = set(input_list)
    occurrence_count = Counter(input_list)

    return list(unique_values), occurrence_count

In [17]:
# Funkcja sprawdza czy peirwszy elemnt tupli jest kluczem do słownika, jeśli tak to dodaje 0 elemnt do listy i zwraca set, który zawiera unikatowe wartości dla danego klucza

def find_all_values_to_dictonary (key: str ,list_of_tuple: list) -> set:
    di = []
    for i in list_of_tuple:
        if i[1] == key:
            di.append(i[0])
    return set(di)


In [18]:
#

from itertools import chain

def get_all_values (dict1: dict, dict0: dict):
    final_dict = {}
    key_lvl1 = []
    key_lvl1.append(list(dict1.keys()))
    key_lvl1 = list(chain(*key_lvl1))
    not_working_keys = []
    for i in key_lvl1:
        temp_vals = []
        temp_keys = dict1[i]
        temp_keys = [x for x in temp_keys if x is not np.nan]
        for j in temp_keys:
            try:
                temp_vals.append(list(dict0[j]))
            except Exception:
                    not_working_keys.append(j)
        temp_vals = set(item for sublist in temp_vals for item in sublist)
        final_dict[i] = temp_vals
    return not_working_keys, final_dict

In [19]:
def get_all_values2(dict1: dict, dict0: dict):
    final_dict = {}
    not_working_keys = []

    for key, values in dict1.items():
        valid_values = [v for v in values if v is not np.nan]
        temp_vals = []

        for v in valid_values:
            try:
                temp_vals.extend(dict0[v])
            except Exception:
                not_working_keys.append(v)

        final_dict[key] = set(temp_vals)

    return not_working_keys, final_dict

In [20]:
def title_classification (dict_of_words: dict, title: list) -> str:
    if len(title) > 0:
        for key, values in dict_of_words.items():
            for i in title:
                if i in values:
                    return key

In [21]:
# Zmiana tytułów na lowercase

all_amz_go['Subject'] = [str(row).lower() for row in all_amz_go['Subject']]
all_cus_go['Subject'] = [str(row).lower() for row in all_cus_go['Subject']]

In [22]:
# przekształcenie stringów w komórkach kolumny tags na listę, dzielone po przecinku 

all_amz_go['Tags'] = [split_tags(row, ',')for row in all_amz_go['Tags']]
all_cus_go['Tags'] = [split_tags(row, ',')for row in all_cus_go['Tags']]

In [23]:
# utworzenie kolumny, w której jest napisane jakiego rynku dotyczy wiadomość, na podstawie kolumny tags

all_amz_go['rynek'] = [get_market(row) for row in all_amz_go['Tags']]
all_cus_go['rynek'] = [get_market(row) for row in all_cus_go['Tags']]

In [24]:
# usunięcie wierszy, które w kolumnie 'rynek' nie maję 'rynek_de'

# all_amz_go.drop(all_amz_go[all_amz_go['rynek'] != 'rynek_de'].index,  inplace=True)
# all_cus_go.drop(all_cus_go[all_cus_go['rynek'] != 'rynek_de'].index,  inplace=True)

In [25]:
# wyciągnięcie numeru zamówień z kolumn subject i description

all_amz_go['Order_number'] = [order_number(row1, row2) for row1, row2 in zip(all_amz_go['Subject'], all_amz_go['Description'])]
all_cus_go['Order_number'] = [order_number(row1, row2) for row1, row2 in zip(all_cus_go['Subject'], all_cus_go['Description'])]

In [26]:
# resetowanie indexów

all_amz_go.reset_index(drop=True, inplace=True)
all_cus_go.reset_index(drop=True, inplace=True)

In [27]:
# zmiana obiektów z kolummny 'Created Time' na daty

from datetime import datetime

format_date = "%d %m %Y %H:%M"

all_amz_go['Created Time'] = [datetime.strptime(row, format_date) for row in all_amz_go['Created Time']] 
all_cus_go['Created Time'] = [datetime.strptime(row, format_date) for row in all_cus_go['Created Time']] 

In [28]:
# wykorzystanie funkcji do splitowania aby utworzyć kolumnę 

all_amz_go['title_list'] = [split_tags(row, ' ') for row in all_amz_go['Subject']]
all_cus_go['title_list'] = [split_tags(row, ' ') for row in all_cus_go['Subject']]

In [29]:
# sprawdzenie czy funcja split_tags zadzaiała poprawnie

all_cus_go['title_list'][1]

['[de]', '302-0014409-7188317', 'niekompletne']

W tym momencie postanowieł sprawdzić, czy zamówienia, z jednej i drugiej tabeli się pokrywają w jakimś najmniejszym stopniu

In [30]:
# za pomocą wcześniej stworzonej funkcji tworzę listę unikatowych numerów ze wszystkich wierszy jednego data set

all_amz_go_list = []
[all_order(row, all_amz_go_list) for row in all_amz_go['Order_number']]
all_amz_go_list = [item for sublist in all_amz_go_list for item in sublist]

In [31]:
# za pomocą wcześniej stworzonej funkcji tworzę listę unikatowych numerów ze wszystkich wierszy drugiego data set

all_cus_go_list = []
[all_order(row, all_cus_go_list) for row in all_cus_go['Order_number']]
all_cus_go_list = [item for sublist in all_cus_go_list for item in sublist]

In [32]:
# zamieniam stworzyonych list na set

mcg_order_set = set(all_amz_go_list)
mag_rynek_de_order_set = set(all_cus_go_list)

In [33]:
# dodanie kontrolnie jednej takiej samej wartości do obu setów

mcg_order_set.add('123')
mag_rynek_de_order_set.add('123')

In [34]:
# porównanie dwóch

y = mcg_order_set.intersection(mag_rynek_de_order_set)

In [35]:
# Wyświetlenie ile mamy takich samych numerów zamówień w jednym i drugim secie

len(y)

11037

Powrót do preprocesingu danych

In [36]:
# weryfickaja działania funkcji do czyszczenia tytułów maili

print(find_words_with_more_than_four_characters2(all_cus_go['Subject'][5552]))

['zapytanie', 'amandine', 'perez']


In [37]:
# utworzenie nowej kolumny label i dodanie do niej słów, które nadają się na labelki

all_cus_go['label'] = [find_words_with_more_than_four_characters2(row) for row in all_cus_go['Subject']]

In [38]:
# Stworzenie listy ze wszystkimi wartościami z kolumny labels

label_list = []
[all_order(row, label_list) for row in all_cus_go['label']]
label_list = [item for sublist in label_list for item in sublist]

In [39]:
# Stworzenie dwóch list na podstawie listy wartości z kolumny['label'], jedna lista to unikatowe wartości, druga lista unikatowe wartości z informacją ile dana wartość występowała

uniq_vales, uniq_values_count = count_occurrences(label_list)

In [40]:
# Stworzenie obiektu spacy z polskim słownikiem

nlp = spacy.load("pl_core_news_lg")

In [41]:
# Zmiana unikatowej listy na string

uniq_vales_sentence = ' '.join(uniq_vales)

In [42]:
# tokenizacja stringu unikatowych wartości z kolumny tags

doc = nlp(uniq_vales_sentence)

In [43]:
# lemantyzacja wcześniej stworzonych tokenów. Stworzenie listy i listy tupli. Lista tupli zawierające słowo i jej podstawę słowotwórczą, sama lista zawiera wszystkie podstwy słowotwórcze 

all_lem = []
all_words = []
for token in doc:
    all_words.append((token.text, token.lemma_))
    all_lem.append(token.lemma_)

In [44]:
# import pickle
# with open('all_lem.pkl', 'wb') as file:
#     pickle.dump(all_lem, file)

In [45]:
# stworzenie słownika ze specjalnego obieku counter

uniq_values_count_dict = dict(uniq_values_count)

In [46]:
# Stworzenie słownika gdzie klucz to podstawa słowotwórcza a values jest listą słów dla tej podstawy występująca w kolumnie tags

lem_dict_0 = {}
for i in set(all_lem):
    lem_dict_0[i] = find_all_values_to_dictonary(i,all_words)

In [47]:
# Stworzenie słownika, gdzie klucz to podstawa słwowtwórcza, a values to suma wartości występowania słów zawiązanych z tą lemą

number_of_occurence = {}
for i in lem_dict_0:
    number_of_occurence_value = []
    for j in lem_dict_0[i]:
        number_of_occurence_value.append(uniq_values_count_dict[j])
    number_of_occurence[i] = sum(number_of_occurence_value)

In [48]:
# Posortowanie słownika zawierającego podstawę słowotwórczą oraz ilość wystąpnień słów z nim powiązanych

number_of_occurence_sorted = sorted(number_of_occurence.items(), key=lambda x:x[1], reverse=True)

In [49]:
labels_df = pd.read_csv('labels2.csv', sep=';', header=None)
labels_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,anulacja,anulować,anulacja,anulowac,anulacj,anulownie,anulownaie,nieanulowanie,anulaj,anuloewanie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,reklamacja,reklamacja,reklamation,reklamcja,reklaamcja,reklamajci,reklmacja,reklacja,reklama,reklamationa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,wysyłka,wysyłce,wysyłka,wysyłke,wysyp,wysylce,wysyłkąć,wysyka,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,refund,refund,refundacja,refunda,refunf,refung,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,przesyłka,przesyłka,przesylka,przesyłewka,przesyłkami,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
labels_df = labels_df.T
labels_df.columns = labels_df.iloc[0]
label_name = labels_df.iloc[0]
labels_df = labels_df[1:]

In [51]:
lem_dict_1 = labels_df.to_dict(orient='list')

In [52]:
not_working_keys, final_dict = get_all_values(lem_dict_1, lem_dict_0)

In [53]:
len(not_working_keys)

60

In [54]:
all_cus_go['final_label'] = [title_classification(final_dict, row) for row in all_cus_go['label']]

In [55]:
df_to_model = pd.DataFrame()
for i in all_cus_go['final_label'].unique():
    subset = all_cus_go[all_cus_go['final_label'] == i].head(1000).reset_index(drop=True)
    df_to_model = pd.concat([df_to_model, subset], axis=0)

In [56]:
df_to_model['final_label'].value_counts()

zapytanie       1000
faktura         1000
anulacja        1000
paczka          1000
wysyłka         1000
dostawa         1000
zwrot           1000
reklamacja      1000
adres            957
brak             798
refund           776
uszkodzony       418
przesyłka        411
niekompletne     395
ponowna          158
inne             127
opóźnienie        81
Name: final_label, dtype: int64

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [58]:
X = df_to_model['Description']
y = df_to_model['final_label']

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [60]:
tfidf = TfidfVectorizer()
tfidf.fit(X_train)

TfidfVectorizer()

In [61]:
X_train_tf = tfidf.transform(X_train)
X_test_tf = tfidf.transform(X_test)

In [62]:
X_train_tf.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [63]:
tfidf.get_feature_names_out()

array(['00', '000', '0000', ..., 'żądany', 'żądanych', 'żę'], dtype=object)

In [64]:
pd.DataFrame(X_train_tf.todense(),
             columns=tfidf.get_feature_names_out())

,00,000,0000,0000000,000000992432,0000028368,0000047237,0000142805,0000357766,0000373427,...,żółty,żółtą,żąda,żądają,żądam,żądania,żądanie,żądany,żądanych,żę
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9691,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train_tf, y_train)

y_pred = lr.predict(X_test_tf)

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       adres       0.80      0.89      0.84       186
    anulacja       0.96      0.89      0.92       185
        brak       0.80      0.77      0.79       154
     dostawa       0.71      0.77      0.74       196
     faktura       0.97      0.97      0.97       208
        inne       0.81      0.54      0.65        24
niekompletne       0.91      0.75      0.82        79
  opóźnienie       0.91      0.45      0.61        22
      paczka       0.88      0.75      0.81       199
     ponowna       0.85      0.62      0.72        37
   przesyłka       0.88      0.67      0.76        84
      refund       0.85      0.84      0.85       146
  reklamacja       0.69      0.80      0.74       208
  uszkodzony       0.78      0.71      0.74        76
     wysyłka       0.75      0.87      0.81       216
   zapytanie       0.85      0.92      0.88       202
       zwrot       0.74      0.74      0.74       203

    accuracy              

In [66]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train_tf, y_train)

y_pred = rfc.predict(X_test_tf)

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       adres       0.76      0.90      0.82       186
    anulacja       0.95      0.90      0.93       185
        brak       0.80      0.76      0.78       154
     dostawa       0.71      0.74      0.72       196
     faktura       0.94      0.96      0.95       208
        inne       0.87      0.54      0.67        24
niekompletne       0.95      0.80      0.87        79
  opóźnienie       0.71      0.45      0.56        22
      paczka       0.87      0.79      0.83       199
     ponowna       0.84      0.70      0.76        37
   przesyłka       0.87      0.71      0.78        84
      refund       0.86      0.86      0.86       146
  reklamacja       0.69      0.78      0.73       208
  uszkodzony       0.76      0.74      0.75        76
     wysyłka       0.85      0.85      0.85       216
   zapytanie       0.87      0.92      0.90       202
       zwrot       0.69      0.69      0.69       203

    accuracy              

In [67]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train_tf, y_train)

y_pred = mnb.predict(X_test_tf)

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       adres       0.84      0.83      0.84       186
    anulacja       0.87      0.88      0.87       185
        brak       0.65      0.47      0.55       154
     dostawa       0.48      0.78      0.59       196
     faktura       0.96      0.95      0.95       208
        inne       0.86      0.50      0.63        24
niekompletne       1.00      0.18      0.30        79
  opóźnienie       0.00      0.00      0.00        22
      paczka       0.87      0.55      0.67       199
     ponowna       0.00      0.00      0.00        37
   przesyłka       1.00      0.36      0.53        84
      refund       0.86      0.73      0.79       146
  reklamacja       0.56      0.83      0.67       208
  uszkodzony       0.88      0.18      0.30        76
     wysyłka       0.55      0.82      0.66       216
   zapytanie       0.91      0.88      0.89       202
       zwrot       0.56      0.75      0.64       203

    accuracy              

spaCy model

In [68]:
from spacy.lang.pt.stop_words import STOP_WORDS
import string

In [69]:
nlp = spacy.blank("pl")

In [70]:
df = pd.DataFrame(df_to_model[['Description', 'final_label']])
df.reset_index(inplace=True)

In [71]:
REGX_URL = r"(https?://|www\.)[^\s/$.?#].[^\s]*"

def preprocessing(text):
  text = text.lower()
  text = text.replace("&quot;", '"')

  text = re.sub(REGX_URL, ' ', text)

  tokens = [token.text for token in nlp(text)]

  tokens = [t for t in tokens if 
              t not in STOP_WORDS and 
              t not in string.punctuation]

  tokens = [t for t in tokens if not t.isdigit()]

  return " ".join(tokens)

In [72]:
df['Description_cleaned'] = df['Description'].apply(preprocessing)

In [73]:
y =pd.get_dummies((df['final_label']), prefix='label')
label = list(y.columns)
y = y.to_dict('index')

In [74]:
dataset = list(zip(df['Description_cleaned'], [{'cats':cats} for cats in y.values()]))
print(dataset[0])

('klient ma pytanie produktu b07r42w8j9 jakie połączenie ma dachówka wentylacyjna link zapytania    pozdrawiam beata', {'cats': {'label_adres': 0, 'label_anulacja': 0, 'label_brak': 0, 'label_dostawa': 0, 'label_faktura': 0, 'label_inne': 0, 'label_niekompletne': 0, 'label_opóźnienie': 0, 'label_paczka': 0, 'label_ponowna': 0, 'label_przesyłka': 0, 'label_refund': 0, 'label_reklamacja': 0, 'label_uszkodzony': 0, 'label_wysyłka': 0, 'label_zapytanie': 1, 'label_zwrot': 0}})


In [75]:
train_data, test_data = train_test_split(dataset,
                                        test_size=0.25,
                                        random_state=42)
dev_data, test_data = train_test_split(test_data, 
                                       test_size=0.4,
                                       random_state=42)

print(f"Total: {len(dataset)} - Train:  {len(train_data)} - Dev: {len(dev_data)} - Test: {len(test_data)}")

Total: 12121 - Train:  9090 - Dev: 1818 - Test: 1213


In [76]:
def convert(data, outfile):
    db = spacy.tokens.DocBin()

    for text, labels in data:
        doc = spacy.tokens.Doc(nlp.vocab, words=text.split())
        doc.cats.update(labels['cats'])
        db.add(doc)

    db.to_disk(outfile)

convert(train_data, "./train.spacy")
convert(dev_data, "./dev.spacy")
convert(test_data, "./test.spacy")

In [77]:
textcat = nlp.add_pipe("textcat_multilabel")
for i in label:
    textcat.add_label(i)

In [78]:
# ! python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [79]:
# ! python -m spacy train config.cfg --paths.train ./train.spacy  --paths.dev ./dev.spacy --output model --verbose

In [80]:
# output_dir = 'C:\\Users\\48575\\email_classification\\spaCy_model'
# nlp.to_disk(output_dir)

In [81]:
saved_model_dir = 'C:\\Users\\48575\\email_classification\\spaCy_model'
nlp = spacy.load(saved_model_dir)

In [82]:
! python -m spacy evaluate ./model/model-best/ ./test.spacy

ℹ Using CPU

================================== Results ==================================

TOK                   -     
TEXTCAT (macro AUC)   94.54 
SPEED                 187704


=========================== Textcat F (per label) ===========================

                          P       R       F
label_adres          100.00   82.35   90.32
label_anulacja        96.51   94.32   95.40
label_brak            92.31   82.19   86.96
label_dostawa         87.50   27.45   41.79
label_faktura         99.04   97.17   98.10
label_inne           100.00   38.89   56.00
label_niekompletne    89.19   89.19   89.19
label_opóźnienie     100.00   72.73   84.21
label_paczka          92.63   81.48   86.70
label_ponowna        100.00   73.91   85.00
label_przesyłka      100.00   76.19   86.49
label_refund          85.53   92.86   89.04
label_reklamacja      95.35   82.00   88.17
label_uszkodzony      93.75   78.95   85.71
label_wysyłka         95.45   19.44   32.31
label_zapytanie       97.44   84.44 

In [91]:
def predict_spaCy (text):
    preprocessed_text = preprocessing(text)
    doc = nlp(preprocessed_text)
    label = max(doc.cats, key=doc.cats.get)
    return label